## Feature Engineering & Aggregation

In [79]:
# Group bool columns into percentages of pos/neg by card_id
bool_cols = [
       'new_merchant_flag','authorized_flag','category_1',
       'category_2_2.0', 'category_2_3.0', 'category_2_4.0',
       'category_2_5.0', 'category_2_NA', 'category_3_B',
       'category_3_C', 'category_3_NA'
]

percentage_cols = encoded_df.groupby('card_id')[bool_cols].apply(lambda x:(x.sum()/len(x)).round(3))
percentage_cols.head(3)

,new_merchant_flag,authorized_flag,category_1,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_2_NA,category_3_B,category_3_C,category_3_NA
card_id,,,,,,,,,,,
C_ID_00007093c1,0.013,0.768,0.185,0.0,0.801,0.0,0.007,0.185,0.841,0.159,0.000
C_ID_0001238066,0.169,0.980,0.027,0.0,0.000,0.0,0.155,0.068,0.730,0.243,0.027
C_ID_0001506ef0,0.015,0.940,0.000,0.0,0.970,0.0,0.000,0.000,0.015,0.000,0.000


In [84]:
# Create a new column with count number of transactions per card_id
date_cols = encoded_df.groupby('card_id').size().reset_index(name='transactions_count')

# Difference between first and last purchase
first_last_purchase = encoded_df.groupby('card_id')['purchase_date'].agg(['min', 'max']).reset_index()
first_last_purchase['days_active'] = (first_last_purchase['max'] - first_last_purchase['min']).dt.days

# merge new features and first_last_purchase on card_id
date_cols = pd.merge(date_cols, first_last_purchase, on='card_id', how='left')

# frequency of transactions per card_id
date_cols['frequency'] = date_cols['transactions_count']/date_cols['days_active']

# Rename min and max columns
date_cols.rename(columns={'min':'first_purchase', 'max':'last_purchase'}, inplace=True)

# reset card_id as index
date_cols.set_index('card_id', inplace=True)

date_cols.head(3)

,transactions_count,first_purchase,last_purchase,days_active,frequency
card_id,,,,,
C_ID_00007093c1,151,2017-02-14 14:00:43,2018-04-09 16:23:59,419,0.360382
C_ID_0001238066,148,2017-09-28 22:25:14,2018-04-30 19:57:30,213,0.694836
C_ID_0001506ef0,67,2017-01-14 16:16:01,2018-03-22 09:14:30,431,0.155452


In [94]:
# Take average values of numerical variables
to_average = ['installments', 'month_lag', 'purchase_amount']
average_cols = encoded_df.groupby('card_id')[to_average].apply(lambda x:(x.sum()/len(x)).round(3))
average_cols.rename(
    columns={
        'purchase_amount' : 'avg_purchase',
        'installments' : 'avg_installments',
        'month_lag' : 'avg_month_lag'
    }, inplace=True)

# Calculate total money spent by user
purchase_amount_sum = encoded_df.groupby('card_id')['purchase_amount'].apply(lambda x:x.sum())

num_cols = pd.merge(average_cols, purchase_amount_sum, on='card_id', how='left')
num_cols.rename(columns={'purchase_amount' : 'sum_purchase'}, inplace=True)
num_cols.head(3)

,avg_installments,avg_month_lag,avg_purchase,sum_purchase
card_id,,,,
C_ID_00007093c1,1.285,-5.748,-0.518,-78.173564
C_ID_0001238066,1.615,-1.277,-0.585,-86.596930
C_ID_0001506ef0,0.015,-4.746,-0.527,-35.333881


In [121]:
# For each card: grab the number of associated ids of each type,
# the number of ids tying for the mode, and one of those ids

aggregate_by = {
    'unique': lambda x: len(x.unique()),
    'mode_len': lambda x: len(x.mode()),
    'mode': lambda x: x.mode().iloc[0]
}

result_df = pd.DataFrame()

for agg in aggregate_by:

    func = aggregate_by[agg]

    temp_df = encoded_df.groupby('card_id').agg(
        {
            'city_id': func,
            'merchant_category_id': func,
            'merchant_id': func,
            'state_id': func,
            'subsector_id': func
        }
    )

    temp_df.rename(columns={
        'city_id': f'city_{agg}',
        'merchant_category_id': f'merchant_category_{agg}',
        'merchant_id': f'merchant_{agg}',
        'state_id': f'state_{agg}',
        'subsector_id': f'subsector_{agg}'
    }, inplace=True)

    if result_df.empty:
        result_df = temp_df
    else:
        result_df = pd.merge(result_df, temp_df, on='card_id', how='left')
    
result_df.head(3)

,city_unique,merchant_category_unique,merchant_unique,state_unique,subsector_unique,city_mode_len,merchant_category_mode_len,merchant_mode_len,state_mode_len,subsector_mode_len,city_mode,merchant_category_mode,merchant_mode,state_mode,subsector_mode
card_id,,,,,,,,,,,,,,,
C_ID_00007093c1,5,19,31,4,13,1,1,1,1,1,244,307,M_ID_9400cf2342,2,19
C_ID_0001238066,19,34,90,6,19,1,1,1,1,1,314,307,M_ID_d17aabd756,9,19
C_ID_0001506ef0,3,19,29,2,12,1,1,1,1,1,137,705,M_ID_b1fc88154d,19,33


In [122]:
# Get which id of each type the user has spent the most money on

city_sum = encoded_df.groupby(['card_id', 'city_id']).agg({
    'purchase_amount': lambda x: (np.abs(x)).sum()
}).reset_index()
max_city = city_sum.groupby('card_id').apply(lambda x: x.loc[x['purchase_amount'].idxmax()])[['city_id']]
max_city.rename(columns={'city_id':'city_max_spent'}, inplace=True)

merchant_category_sum = encoded_df.groupby(['card_id', 'merchant_category_id']).agg({
    'purchase_amount': lambda x: (np.abs(x)).sum()
}).reset_index()
max_merchant_category = merchant_category_sum.groupby('card_id').apply(lambda x: x.loc[x['purchase_amount'].idxmax()])[['merchant_category_id']]
max_merchant_category.rename(columns={'merchant_category_id':'merchant_category_max_spent'}, inplace=True)

merchant_sum = encoded_df.groupby(['card_id', 'merchant_id']).agg({
    'purchase_amount': lambda x: (np.abs(x)).sum()
}).reset_index()
max_merchant = merchant_sum.groupby('card_id').apply(lambda x: x.loc[x['purchase_amount'].idxmax()])[['merchant_id']]
max_merchant.rename(columns={'merchant_id':'merchant_max_spent'}, inplace=True)

state_sum = encoded_df.groupby(['card_id', 'state_id']).agg({
    'purchase_amount': lambda x: (np.abs(x)).sum()
}).reset_index()
max_state = state_sum.groupby('card_id').apply(lambda x: x.loc[x['purchase_amount'].idxmax()])[['state_id']]
max_state.rename(columns={'state_id':'state_max_spent'}, inplace=True)

subsector_sum = encoded_df.groupby(['card_id', 'subsector_id']).agg({
    'purchase_amount': lambda x: (np.abs(x)).sum()
}).reset_index()
max_subsector = subsector_sum.groupby('card_id').apply(lambda x: x.loc[x['purchase_amount'].idxmax()])[['subsector_id']]
max_subsector.rename(columns={'subsector_id':'subsector_max_spent'}, inplace=True)

# Merge the results
merged_df = pd.merge(max_city, max_merchant_category, on='card_id', how='inner')
merged_df = pd.merge(merged_df, max_state, on='card_id', how='inner')
merged_df = pd.merge(merged_df, max_merchant, on='card_id', how='inner')
merged_df = pd.merge(merged_df, max_subsector, on='card_id', how='inner')
merged_df.head(3)


,city_max_spent,merchant_category_max_spent,state_max_spent,merchant_max_spent,subsector_max_spent
card_id,,,,,
C_ID_00007093c1,244,307,2,M_ID_9400cf2342,19
C_ID_0001238066,314,307,9,M_ID_d17aabd756,19
C_ID_0001506ef0,137,705,19,M_ID_b1fc88154d,33


In [127]:
# Merge all of the aggregations
final_df = pd.merge(num_cols, result_df, on='card_id', how='inner')
final_df = pd.merge(final_df, merged_df, on='card_id', how='inner')
final_df = pd.merge(final_df, percentage_cols, on='card_id', how='inner')
final_df = pd.merge(final_df, date_cols, on='card_id', how='inner')

In [125]:
# Convert int64 to int16
int64_columns = final_df.select_dtypes(include=['int64']).columns
final_df[int64_columns] = final_df[int64_columns].astype('int16')

In [126]:
final_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 325540 entries, C_ID_00007093c1 to C_ID_fffffd5772
Data columns (total 41 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   avg_installments             325540 non-null  float64       
 1   avg_month_lag                325540 non-null  float64       
 2   avg_purchase                 325540 non-null  float64       
 3   sum_purchase                 325540 non-null  float64       
 4   city_id                      325540 non-null  int16         
 5   city_unique                  325540 non-null  int16         
 6   merchant_category_unique     325540 non-null  int16         
 7   merchant_unique              325540 non-null  int16         
 8   state_unique                 325540 non-null  int16         
 9   subsector_unique             325540 non-null  int16         
 10  city_mode_len                325540 non-null  int16         
 11  merchant

In [129]:
final_df.to_csv('./Data/cleaned_data.csv', index=False)
final_df.head(3)

,avg_installments,avg_month_lag,avg_purchase,sum_purchase,city_id,city_unique,merchant_category_unique,merchant_unique,state_unique,subsector_unique,...,category_2_5.0,category_2_NA,category_3_B,category_3_C,category_3_NA,transactions_count,first_purchase,last_purchase,days_active,frequency
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,1.285,-5.748,-0.518,-78.173564,244,5,19,31,4,13,...,0.007,0.185,0.841,0.159,0.000,151,2017-02-14 14:00:43,2018-04-09 16:23:59,419,0.360382
C_ID_0001238066,1.615,-1.277,-0.585,-86.596930,314,19,34,90,6,19,...,0.155,0.068,0.730,0.243,0.027,148,2017-09-28 22:25:14,2018-04-30 19:57:30,213,0.694836
C_ID_0001506ef0,0.015,-4.746,-0.527,-35.333881,137,3,19,29,2,12,...,0.000,0.000,0.015,0.000,0.000,67,2017-01-14 16:16:01,2018-03-22 09:14:30,431,0.155452
